Code Challenge (3 points): Now that you have seen how GibbsSampler works, implement it in Python as a final challenge in this chapter. Your function should take a parameter N corresponding to the number of iterations that we plan to run the program. Don't forget to use pseudocounts!

Note: If you have trouble implementing GibbsSampler, please see below, where we provide a description of GibbsSampler using pseudocode, a method of describing algorithms that is not dependent on a particular programming language like Python. If you are interested in learning more about how biological problems can be solved computationally, please check out our Bioinformatics Specialization or its print companion, Bioinformatics Algorithms: An Active Learning Approach, which uses pseudocode to discuss dozens of bioinformatics algorithms.


In [37]:
from numba import jit
import random

# Input: A dictionary Probabilities, where keys are k-mers and values are the probabilities of these k-mers (which do not necessarily sum up to 1)
# Output: A normalized dictionary where the probability of each k-mer was divided by the sum of all k-mers' probabilities
#@jit
def Normalize(Probabilities):
    _sum_probs = 0.0 #Sum of all probabilities
    new_prob = Probabilities.copy()#Make a copy.
    
    for key in Probabilities.keys():
        _sum_probs += Probabilities[key] #add up all probabilites
        
    for key in Probabilities.keys():
        new_prob[key] = Probabilities[key] / _sum_probs   #Normalize values   
        
    return new_prob

#@jit
def WeightedDie(Probabilities):
    key_list = Probabilities.keys()
    bounded_probabilities = Probabilities.copy()
    #print ("Probabilities Raw", Probabilities)
    #Build an ascending list of values
    previous_value = 0.0 #Start with 0.0 Prob
    for key in key_list:
        bounded_probabilities[key] = previous_value + Probabilities[key]
        previous_value = bounded_probabilities[key]
    
    #print ("Bounded Probab", bounded_probabilities)
    #Now we roll the dice to see which bound we fall in.
    previous_value = 0.0 #Start with 0.0 Prob
    rand_value = random.uniform(0,1) 
    for key in key_list:
        #If our choice is between 0.0 and 0.25, then return 'A', for example
        if (rand_value >= previous_value) and (rand_value < bounded_probabilities[key]):
            return key
        previous_value = bounded_probabilities[key] #We try the next value

#@jit
def Pr(Motif, Profile):
    prob=1.0
    for char in range(len(Motif)):
        prob *= Profile[Motif[char]][char] # What is the probability that this character is in this position?    
    #print ("Prob in Pr", prob)
    return prob

# Input:  A string Text, a profile matrix Profile, and an integer k
# Output: ProfileGeneratedString(Text, profile, k)
#@jit
def ProfileGeneratedString(Text, profile, k):
    n = len(Text)
    probabilities = {} 
    for i in range(n-k+1):
        #print("Pr", Pr(Text[i:i+k], profile))
        probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
    probabilities = Normalize(probabilities)
    #print (probabilities)
    return WeightedDie(probabilities)    

#@jit
def Score(Motifs):
    #This really seems inefficient. We could have consensus return both values...
    counts = CountWithPseudocounts(Motifs)
    consensus, counts = Consensus(Motifs)
    score = 0
    for char in range(len(consensus)):
        nucleotide = consensus[char] #Our nucleoutide
        keys = [key for key in ['A','C',"G","T"] if key != nucleotide ] # What are the non-consensus nucleotides
        for key in keys:# For each key
            score += counts[key][char] # add the number of times we were incorrect to our score
    return score

# Input:  A set of kmers Motifs
# Output: CountWithPseudocounts(Motifs)
#@jit
def CountWithPseudocounts(Motifs):
    count = {} # initializing the count dictionary
    #Initialize each nucleotide with an empty list, 
    for nucleotide in ["A","C","G","T"]:
        count[nucleotide] = []     
    for ind in range(len(Motifs[0])):
        for nucleotide in ["A","C","G","T"]:
            count[nucleotide].append(1.0) #everything must have a 1 initially  (Now that we are dealing with pseudo counts)
        for motif in range(len(Motifs)): #For each Motif, loop through chars
            count[Motifs[motif][ind]][ind] += 1.0 # FOr each nuc, increment its count for that Motif
    return count

#@jit
# Input:  A list of strings Dna, and integers k and t
# Output: RandomMotifs(Dna, k, t)
def RandomMotifs(Dna, k, t):
    rand_motifs = []
    m = len(Dna[0])# What is the max length of one of our strings?
    for text in Dna:
            i = random.randint(0, m-k)#Pick a random int that is up to k less than the max value
            rand_motifs.append(text[i:i+k])
    return rand_motifs

#@jit
def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {}
    counts = CountWithPseudocounts(Motifs)

    for nucleotide in ["A","C","G","T"]:
        #everything must divided by the total number of Motifs  
        #We added a single count to each nucleotide, hence the +4.0
        profile[nucleotide] = [ float(count)/(float(t) + 4.0) for count in counts[nucleotide]]  
    return profile

# Input:  A set of kmers Motifs
# Output: A consensus string of Motifs.
def Consensus(Motifs):
    consensus = "" #empty string.
    counts = CountWithPseudocounts(Motifs)

    for j in range(len(Motifs[0])):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if counts[symbol][j] > m:
                m = counts[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol # Add most frequent symbol
    return consensus, counts

```
    GibbsSampler(Dna, k, t, N)
        randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
        ﻿BestMotifs ← Motifs
        for j ← 1 to N
            i ← randomly generated integer between 1 and t
            Profile ← profile matrix formed from all strings in Motifs except for Motifi
            Motif_i ← Profile-randomly generated k-mer in the i-th string
            if Score(Motifs) < Score(BestMotifs)
                BestMotifs ← Motifs
        return BestMotifs
        
```

In [45]:
#@jit(nopython=True)
def GibbsSampler(Dna, k, t , N):
    BestMotifs = RandomMotifs(Dna, k, t)
    for iter_num in range(N):
        rand_value = random.randint(0,t)  # Which kmer will we leave untouched
        _minus1_Motifs = []
        for x in range(t):
            if x != rand_value:
                _minus1_Motifs.append(BestMotifs[x])
        Profile = ProfileWithPseudocounts(_minus1_Motifs) #Build profile from this subset
        #print ("Profile",Profile)
        #Build t motifs with this profile
        Motifs = []
        for i in range(t):
            #print (i, ProfileGeneratedString(Dna, Profile, k))
            Motifs.append(ProfileGeneratedString(Dna[i], Profile, k)) 
        #print (Motifs, BestMotifs)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [ ]:
k, t, N = 8, 5, 100
Dna1 =[ "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
# Possible Output: AACGGCCA AAGTGCCA TAGTACCG  AAGTTTCA ACGTGCAA

In [40]:
import time
t0 = time.time()
best_out = GibbsSampler(Dna1, k, t , N)
print (time.time()-t0)
print (best_out)

0.03390836715698242
['GTAAACGG', 'GTATGTGT', 'GTATACAG', 'GTGCACGT', 'CTCCACGT']


In [46]:
import time
t0 = time.time()
best_out = GibbsSampler(Dna1, k, t , 10000)#2.972266674041748
print (time.time()-t0)
print (best_out)

3.046574831008911
['GGGTGTTC', 'ATGTGTAA', 'AAGTATAC', 'AGGTGCAC', 'ACGTGCAA']


Exercise Break (1 point): Run GibbsSampler on the DosR dataset (click here to download) for 20 different starts, each time with N = 100, and with k-mer length equal to 15. Don't forget to use pseudocounts!

In [42]:
Dna = [
    "GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC",
    "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG",
    "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC",
    "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC",
    "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG",
    "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA",
    "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA",
    "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG",
    "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG",
    "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"
] 

In [ ]:
# import the random package

# Copy your GibbsSampler function (along with all required subroutines) below this line

# Copy the ten strings occurring in the hyperlinked DosR dataset below.
Dna = [
    "GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC",
    "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG",
    "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC",
    "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC",
    "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG",
    "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA",
    "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA",
    "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG",
    "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG",
    "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"
] 

# set t equal to the number of strings in Dna, k equal to 15, and N equal to 100
t , k, N = len(Dna), 15, 10000
# Call GibbsSampler(Dna, k, t, N) 20 times and store the best output in a variable called BestMotifs
BestMotifsScore=-999
for i in range(20):
    _motifs = GibbsSampler(Dna, k, t, N)
    if Score(_motifs) > BestMotifsScore:
        BestMotifsScore=Score(_motifs)
        BestMotifs=_motifs
# Print the BestMotifs variable
print (BestMotifs)
# Print Score(BestMotifs)
print (BestMotifsScore)

Searching for the DosR motif
What k-mer size should we choose in order to analyze the DosR dataset using GibbsSampler? Taking a wild guess and running RandomizedMotifSearch for varying values of k returns the consensus strings shown in the figure below.



Figure: The result of running RandomizedMotifSearch for k between 8 and 12 as well as k equal to 20. Consensus strings have been aligned in order to highlight their similarities.

STOP and Think: Can you infer the DosR binding site from these strings? What do you think is the length of the binding site?

The motif of length 20 found by RandomizedMotifSearch is "CGGGACCTACGTCCCTAGCC" (with score 57). In 2000 runs, GibbsSampler generated a different collection of motifs with a smaller score of 55. But these motifs had the same consensus string!

Despite evidence in favor of this consensus string as the DosR motif, we are still not sure that it should have length 20, or even that it is correct, since some other motif finding algorithm might find a set of motifs with even lower score. Can you further refine the algorithms that we have presented to find all putative DosR motifs in MTB as well as all genes that they regulate?

We hope that you have enjoyed learning a little more about how computation is vital to modern biology, and that we have helped you start a path toward becoming an expert programmer. We would love to have you in the Bioinformatics Specialization, an entire series of courses at the frontier of computational biology; if you're interested, please see the Specialization's wacky introductory video at the bottom of the page :)

Even more importantly, you don't have to program in order to complete the Bioinformatics Specialization. We employ pseudocode, a powerful paradigm that helps us discuss algorithms without relying on any particular programming language. If you are interested in continuing on with implementing algorithms, then we provide an "Honors Track" in which you can flex your programming muscles. The first course in the series, Finding Hidden Messages in DNA, covers much of the same introductory material that we have seen in this class, in the hope of making your transition to the Bioinformatics Specialization as easy as possible. 

And if you plan on still using Python, you should be ready to complete the remainder of the Codecademy Python Track. In addition, you now have two files (Replication.py and Motifs.py) full of functions to get you started with running Python on your own machine. To do so, download the latest version of Python 3 from the Python website. We also suggest downloading a development environment like PyCharm that will make building your own projects in Python a breeze.

Happy Coding!